In [1]:
import os 
import torch 
import torch.nn as nn 
import torch.nn.functional as F

# Data

In [2]:
with open(os.path.join(".", "data", "input.txt" ), "r", encoding="utf-8") as f: 
    text = f.read()

In [3]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [4]:
# get all unqiue characters in the set 

unique_chars = sorted(list(set(text)))
vocab_size = len(unique_chars)
print(''.join(unique_chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


# Tokenization

In [5]:
stoi = {ch:i for i, ch in enumerate(unique_chars)}
itos = {i:ch for i, ch in enumerate(unique_chars)}

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [6]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype) 
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [7]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [8]:
block_size = 8
batch_size = 4
x = train_data[:block_size]
y = train_data[1:block_size + 1]

for t in range(block_size): 
    content = x[:t + 1]
    target = y[t]
    print(f"when input is: {content}, target is: {target}")

when input is: tensor([18]), target is: 47
when input is: tensor([18, 47]), target is: 56
when input is: tensor([18, 47, 56]), target is: 57
when input is: tensor([18, 47, 56, 57]), target is: 58
when input is: tensor([18, 47, 56, 57, 58]), target is: 1
when input is: tensor([18, 47, 56, 57, 58,  1]), target is: 15
when input is: tensor([18, 47, 56, 57, 58,  1, 15]), target is: 47
when input is: tensor([18, 47, 56, 57, 58,  1, 15, 47]), target is: 58


In [9]:
torch.manual_seed(1337)
def get_batch(split): 
    data = train_data if split == "train" else val_data

    idx = torch.randint(len(data) - batch_size, (batch_size,))

    x = torch.stack([data[i:i+block_size] for i in idx])
    y = torch.stack([data[i+1:i+block_size+1] for i in idx])

    return x, y

xb, yb = get_batch("train")
print(f"inputs: {xb.shape} || targets: {yb.shape}")

for b in range(batch_size): 
    for t in range(block_size): 
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is: {context.tolist()}, target is: {target}")

inputs: torch.Size([4, 8]) || targets: torch.Size([4, 8])
when input is: [1], target is: 60
when input is: [1, 60], target is: 39
when input is: [1, 60, 39], target is: 47
when input is: [1, 60, 39, 47], target is: 50
when input is: [1, 60, 39, 47, 50], target is: 1
when input is: [1, 60, 39, 47, 50, 1], target is: 63
when input is: [1, 60, 39, 47, 50, 1, 63], target is: 53
when input is: [1, 60, 39, 47, 50, 1, 63, 53], target is: 59
when input is: [46], target is: 43
when input is: [46, 43], target is: 39
when input is: [46, 43, 39], target is: 60
when input is: [46, 43, 39, 60], target is: 43
when input is: [46, 43, 39, 60, 43], target is: 52
when input is: [46, 43, 39, 60, 43, 52], target is: 1
when input is: [46, 43, 39, 60, 43, 52, 1], target is: 44
when input is: [46, 43, 39, 60, 43, 52, 1, 44], target is: 53
when input is: [1], target is: 46
when input is: [1, 46], target is: 43
when input is: [1, 46, 43], target is: 56
when input is: [1, 46, 43, 56], target is: 43
when input is

# Model

In [17]:
class BigramLanguageModel(nn.Module): 
    def __init__(self, vocab_size): 
        super().__init__()

        self.token_embeddings_table = nn.Embedding(vocab_size, vocab_size) 

    def forward(self, idx, targets=None): 
        logits = self.token_embeddings_table(idx) 

        if targets == None: 
            loss = None
        else: 
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(-1)

            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens): 
        for _ in range(max_new_tokens): 
            logits, loss = self(idx) 
            logits = logits[:, -1, :]

            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)

        return idx

In [19]:
model = BigramLanguageModel(vocab_size=vocab_size)
logits, loss = model(xb, yb) 
print(logits.shape)
print(loss)

idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(model.generate(idx, max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.6110, grad_fn=<NllLossBackward0>)

phIOWd3AqNcgg,G!;j
UtVYwJteWJc3xq.NBpFdLXaqK; eyjnB,Icl'Vn3M3M:JSe;bVbN N&DsRi?!DSaeyNZlSYjVkCzkSdoc


In [20]:
# optimizer 
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [23]:
batch_size = 32 

for steps in range(10000): 
    xb, yb = get_batch('train')

    logits, loss = model(xb, yb) 

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.4712576866149902


In [25]:
print(decode(model.generate(idx=torch.zeros((1, 1), dtype=torch.long), max_new_tokens=400)[0].tolist()))


3.
Isch dg.
INGA:
Why chey upess:
AMat has?
ALONTho ie she tot y tord frerd? wh n machy, rave wheas myonann a y! ren:
Thenot hee hestt wor RYourkndiceanche s wie'cco ff s haMys itathey dome?
PHAnd IS:

Wh t fo'soos ouinu atoreng the ICind:
ABOUThaset 'd.
HORThomot Thond y; pin she thow.

VO: g;
Whame ome f icou p
FORinesty ir oun IOUThuess bes llle:
WBr, is llly s ftoonomas til ad Ane w; ine,
THes


# Self attention

In [38]:
B = 2 
T = 4 
C = 32

x = torch.randn(B, T, C)

# a head of self attention 
head_size = 16
key = nn.Linear(C, head_size)
query = nn.Linear(C, head_size)
value = nn.Linear(C, head_size)

k = key(x) 
q = query(x) 
v = value(x) 
wei = q @ k.transpose(-2, -1)

tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, float("-inf"))
wei = F.softmax(wei, dim=-1)

v = value(x) # (B, T, 16)
out = wei @ v  
# out = wei @ x 

out.shape

torch.Size([2, 4, 16])

In [39]:
wei

tensor([[[1.0000, 0.0000, 0.0000, 0.0000],
         [0.0496, 0.9504, 0.0000, 0.0000],
         [0.1834, 0.7755, 0.0411, 0.0000],
         [0.0279, 0.1351, 0.0723, 0.7647]],

        [[1.0000, 0.0000, 0.0000, 0.0000],
         [0.6811, 0.3189, 0.0000, 0.0000],
         [0.7529, 0.2042, 0.0429, 0.0000],
         [0.3278, 0.2006, 0.4157, 0.0559]]], grad_fn=<SoftmaxBackward0>)